In [1]:
from googleapiclient.discovery import build
from oauth2client.service_account import ServiceAccountCredentials

from google.cloud import bigquery
from os import environ


In [2]:
def getGaConnection():
    scopes = ['https://www.googleapis.com/auth/analytics.readonly']
    # your credentials path
    keyFileLocation = './credentials.json'
    credentials = ServiceAccountCredentials.from_json_keyfile_name(keyFileLocation, scopes)

    return build('analyticsreporting', 'v4', credentials=credentials)

In [3]:
def getBqConnection ():
    # your credentials path
    environ["GOOGLE_APPLICATION_CREDENTIALS"] = './credentials.json'
    return bigquery.Client()

In [27]:
def getDataFromGa (viewId,startDate,endDate,metrics,dimensions):
    
    metrics  = [{'expression': metricsName} for metricsName in metrics]
    dimensions  = [{'name': dimensionName} for dimensionName in dimensions]
    
    
    analyticsData = analytics.reports().batchGet(
          body={
            'reportRequests': [
            {
              'viewId': viewId,
              'dateRanges': [{'startDate': startDate, 'endDate': endDate}],
              'metrics': metrics,
              'dimensions': dimensions
            }]
          }
      ).execute()
    return analyticsData

In [22]:
def createTable (tableName,client):
    schema = [

        bigquery.SchemaField('pagePath', 'STRING', mode='REQUIRED'),
        bigquery.SchemaField('channelGrouping', 'STRING', mode='REQUIRED'),
        bigquery.SchemaField('medium', 'STRING', mode='REQUIRED'),
        bigquery.SchemaField('source', 'STRING', mode='REQUIRED'),
        bigquery.SchemaField('referralPath', 'STRING', mode='REQUIRED'),
        bigquery.SchemaField('fullReferrer', 'STRING', mode='REQUIRED'),
        bigquery.SchemaField('sessions', 'INTEGER', mode='REQUIRED'),
        bigquery.SchemaField('pageViews', 'INTEGER', mode='REQUIRED'),

    ]
    table = bigquery.Table(tableName, schema=schema)
    client.create_table(table)

In [23]:
# Initializes a Google Analytics Reporting API V4 service object
analytics = getGaConnection()

# Instantiate the bigquery.Client class to create the BigQuery client
bqClient = getBqConnection()

In [28]:
viewId = 'your_view_id'
startDate = '2019-01-01'
endDate = '2019-10-10'

# Names of the metrics you want to fetch
metrics = [ 'ga:sessions','ga:pageViews']

# Names of the dimensions you want to fetch
dimensions = ['ga:dateHourMinute', 'ga:pagePath','ga:channelGrouping', 'ga:medium', 
              'ga:source','ga:referralPath','ga:fullReferrer'] 

# getting data from google analytics account
analyticsData  = getDataFromGa (viewId,startDate,endDate,metrics,dimensions)

In [29]:
# rearranging data for google bigquery
insertIntoRow = [row['dimensions']+row['metrics'][0]['values']  for row in analyticsData['reports'][0]['data']['rows']]

In [ ]:
projectId = 'your_project_id'
dataset = 'your_dataset'
table = 'your_table'

tablePath = projectId + '.' + dataset  + '.' + table

try :
    # getting the desire table , if table is not present
    # it'll rise a exception
    table = bqClient.get_table(tablePath)
except :
    # creating the table
    createTable(tablePath,bqClient)
    table = bqClient.get_table(tablePath)
    # google need some time to store data after creating a table
    from time import sleep
    sleep(60)

# inserting data in table
bqClient.insert_rows(table, insertIntoRow) 
